# Process

## 0. Dependency Parsing

In [11]:
import spacy
nlp=spacy.load('en_core_web_sm/en_core_web_sm-3.4.1/')

text='I want to eat a apple'

list_text = []
for index,token in enumerate(nlp(text)):
    print(index, token.text,'=>',token.dep_,'=>',token.head.text)
    list_text.append((index,token.text)) 

0 I => nsubj => want
1 want => ROOT => want
2 to => aux => eat
3 eat => xcomp => want
4 a => det => apple
5 apple => dobj => eat


In [12]:
list_text

[(0, 'I'), (1, 'want'), (2, 'to'), (3, 'eat'), (4, 'a'), (5, 'apple')]

## 1. Stack : Enqueue and Dequeue 

In [62]:
from collections import deque
class Stack(object):
    def __init__(self):
        self.container = deque()
    def push(self,value):
        self.container.append(value)
    def pop(self):
        if len(self.container) != 0:
            return self.container.pop()  
        else: raise IndexError("An empty deque")
    # def peek(self):
    #     return  self.container[-1]
    # def is_empty(self):
    #     return len(self.container)==0
    def size(self):
        return len(self.container)

In [63]:
s = Stack()
#push items
for i in range(len(list_text)):
    s.push(list_text[i][1])
    
print("Initial queue")
print(s.container)

#pop items
for i in range(len(list_text)):
    s.pop()
    
print("\nQueue after removing elements")
print(s.container)

Initial queue
deque(['I', 'want', 'to', 'eat', 'a', 'apple'])

Queue after removing elements
deque([])


## 2. Transition-based : Stack and Buffer

In [74]:
class Configuration(object):
    def __init__(self,depency_list):
        self.stack = [0]
        self.buffer = list(range(1,len(depency_list)))
        self.arcs = []
    def __str__(self):
        return f'Stack : {self.stack} Buffer : {self.buffer} Arcs : {self.arcs}'

class Transition(object):
    # Define set of transitions
    LEFT_ARC = "LEFTARC"
    RIGHT_ARC = "RIGHTARC"
    SHIFT = "SHIFT"
    REDUCE = "REDUCE"
    def __init__(self):
        pass
    def left_arc(self,config):
        pass
    def right_arc(self,config):
        pass
    def shift(self,config):
        if len(config.buffer) <= 0:
            return -1
    def reduce(self,config):
        self.state.pop()
    

In [76]:
config = Configuration(list_text)
print(config)

Stack : [0] Buffer : [1, 2, 3, 4, 5] Arcs : []


### Reference : Transition-Based
- https://www.youtube.com/watch?v=xG0MXIPVUdk&list=PLyyEwPZh6aHpDieqWwr7hTgreKnYSY9Ox&index=60
- https://www.youtube.com/watch?v=2jLk93iIyrw&list=PLr9TFf9GjancAHiiP5cqGmYsZad99Uf_h&index=39
- https://www.youtube.com/watch?v=f5-hTA9hA3s&list=WL&index=8
- https://www.youtube.com/watch?v=oLHnqGmQtI4

## 3. Chu-Liu-Edmonds : Maximum Spaning Tree

## 4. Convert to Binary Tree

In [5]:
class Node(object):
    def __init__(self, key):
        self.left  = None
        self.right = None
        self.key = key
        
    def insert(self, key):
        if(self.key):
            if(key < self.key):
                if(self.left == None):
                    self.left = Node(key)
                else:
                    self.left.insert(key)
            
            elif(key >= self.key):
                if(self.right == None):
                    self.right = Node(key)
                else:
                    self.right.insert(key)
        else:
            self.key = key
    
    def printT(self):        
        if (self.left):
            self.left.printT()
        print(self.key)
        if (self.right):
            self.right.printT()
    
    def delete(self,value):
        if value < self.key: #CaseD
            if self.left:
                self.left = self.left.delete(value)
        elif value > self.key: #CaseC
            if self.right:
                self.right = self.right.delete(value)
        else:
            if self.left == None and self.right == None: #Case A and B
                return None
            elif self.left == None:
                return self.right
            elif self.right == None: 
                return self.left

In [13]:
root = Node(len(list_text))
for i in list_text:
    root.insert(i[0])

root.printT()

1
2
3
4
5
6


In [ ]:
def 